# จําลองข้อมูลขนาดเล็กมาทําให้เห็นชัดๆก่อน
เริ่มจากเราต้อง import ไลบราลีที่ต้องการทํากับข้อมูลก่อน 2 ตัวคือ

In [57]:
import numpy as np
import pandas as pd

สร้างข้อมูล 5 ตัวอย่างและมี 3 คอลัมภ์ ในอาร์เรย์ แล้วเก็บใส่ DataFrame

In [58]:
X = pd.DataFrame( np.array(['M','O-','medium','M','O-','high','F','O+','high','F','AB','low','F','B+',np.NaN,]).reshape((5,3)))
X.columns = ['sex','blood_type','edu_level']
print(X)

  sex blood_type edu_level
0   M         O-    medium
1   M         O-      high
2   F         O+      high
3   F         AB       low
4   F         B+       nan


# มาทํา Ordinal Encode กันเถอะ
เริ่มจากเราต้อง import ไลบราลี OrdinalEncoder สําหรับการเข้ารหัสตัวเลข

In [59]:
from sklearn.preprocessing import OrdinalEncoder

ทําการสร้าง instance ของคลาส OrdinalEncoder ก่อน แล้วเรียก fit_transform()

In [60]:
encoder = OrdinalEncoder()
X['new_edu_level'] = encoder.fit_transform(X.edu_level.values.reshape(-1,1))
print(X)

  sex blood_type edu_level  new_edu_level
0   M         O-    medium            2.0
1   M         O-      high            0.0
2   F         O+      high            0.0
3   F         AB       low            1.0
4   F         B+       nan            3.0


ปัญหาคือ NaN มีค่ามากกว่า ระดับการศึกษาอื่น แบบนี้ไม่ถูกต้อง เรามาแก้ไขกัน

# กําหนด category ก่อน encode
กลับไปสร้างข้อมูลเหมือนในหน้า 5 ก่อนนะ แล้วค่อยสร้าง category โดยเรียงลําดับ

In [61]:
cat = pd.Categorical(X.edu_level,categories=['missing', 'low','medium', 'high'],ordered=True)
print(cat)

['medium', 'high', 'high', 'low', NaN]
Categories (4, object): ['missing' < 'low' < 'medium' < 'high']


# แปลงข้อมูล NaN เป็น ‘missing’ ก่อน
ใน category เราระบุ ‘missing’ ไว้ ดังนั้นเราแปลงข้อมูล NaN เป็น ‘missing’

In [62]:
cat = cat.fillna('missing')
print(cat)

['medium', 'high', 'high', 'low', 'missing']
Categories (4, object): ['missing' < 'low' < 'medium' < 'high']


# ทําการแปลงข้อมูลประเภทเป็นตัวเลข
กําหนดตัวเลขให้กับ category data ตาม category ที่เรากําหนดก่อนหน้า

In [63]:
labels, unique = pd.factorize(cat, sort=True)
X.edu_level = labels
print(X)

  sex blood_type  edu_level  new_edu_level
0   M         O-          2            2.0
1   M         O-          3            0.0
2   F         O+          3            0.0
3   F         AB          1            1.0
4   F         B+          0            3.0


# มาทํา OneHot Encode กันเถอะ
เริ่มจากเราต้อง import ไลบราลี OneHotEncoder สําหรับการเข้ารหัสตัวเลข

In [64]:
from sklearn.preprocessing import OneHotEncoder

ทําการสร้าง instance ของคลาส OneHotEncoder ก่อน แล้วเรียก fit_transform()

In [76]:
onehot = OneHotEncoder(dtype=np.int64, sparse_output=True)
nominals = pd.DataFrame(
 onehot.fit_transform(X[['sex', 'blood_type']])\
 .toarray(),
 columns=['F', 'M', 'AB', 'B+','O+', 'O-'])
nominals['edu_level'] = X.edu_level
print(nominals)

   F  M  AB  B+  O+  O-  edu_level
0  0  1   0   0   0   1          2
1  0  1   0   0   0   1          3
2  1  0   0   0   1   0          3
3  1  0   1   0   0   0          1
4  1  0   0   1   0   0          0


ในเวอร์ชันล่าสุดของ scikit-learn ตัวพารามิเตอร์ sparse ได้ถูกเปลี่ยนชื่อเป็น sparse_output คุณจะต้องอัปเดตโค้ดของคุณให้ใช้ sparse_output แทน sparse

In [65]:
onehot = OneHotEncoder(dtype=int,  sparse_output=True)
nominals = pd.DataFrame(
    onehot.fit_transform(X[['sex', 'blood_type']]).toarray(),
    columns=['F', 'M', 'AB', 'B+','O+', 'O-'])
nominals['edu_level'] = X.edu_level
print(nominals)

   F  M  AB  B+  O+  O-  edu_level
0  0  1   0   0   0   1          2
1  0  1   0   0   0   1          3
2  1  0   0   0   1   0          3
3  1  0   1   0   0   0          1
4  1  0   0   1   0   0          0


เกิด column ตามที่เรากําหนด
F , M
AB, B+, O+, O

# Method 1: Using Pandas and Numpy

In [66]:
import seaborn as sns
import pandas as pd
import numpy as np

data = sns.load_dataset('iris')
print('Original Dataset')
data.head()

Original Dataset


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [67]:
# Min-Max Normalization
df = data.drop('species', axis=1)
df_norm = (df-df.min())/(df.max()-df.min())

df_norm = pd.concat((df_norm, data.species), axis=1)
print("Scaled Dataset Using Pandas")
df_norm.head() 

Scaled Dataset Using Pandas


,sepal_length,sepal_width,petal_length,petal_width,species
0,0.222222,0.625000,0.067797,0.041667,setosa
1,0.166667,0.416667,0.067797,0.041667,setosa
2,0.111111,0.500000,0.050847,0.041667,setosa
3,0.083333,0.458333,0.084746,0.041667,setosa
4,0.194444,0.666667,0.067797,0.041667,setosa


# Method 2: Using MinMaxScaler from sklearn

In [68]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
data = sns.load_dataset('iris')
print('Original Dataset')
data.head()

Original Dataset


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [69]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=[
'sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
print("Scaled Dataset Using MinMaxScaler")
df_scaled.head()

Scaled Dataset Using MinMaxScaler


,sepal_length,sepal_width,petal_length,petal_width
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667


# Standardization

In [70]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
data = sns.load_dataset('iris')
print('Original Dataset')
data.head()
std_scaler = StandardScaler()
df_scaled = std_scaler.fit_transform(df.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=[
'sepal_length','sepal_width','petal_length','petal_width'])
print("Scaled Dataset Using StandardScaler")
df_scaled.head()

Original Dataset
Scaled Dataset Using StandardScaler


,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444
